## Feature Extraction

In [10]:
import os
import cv2
import pandas as pd
from radiomics import featureextractor

In [11]:
image_dir = os.path.join(os.getcwd(), "data", "scale_1", "512", "image")
mask_dir = os.path.join(os.getcwd(), "data", "scale_1", "512", "mask")

# Path for the new folder to back up old masks
new_mask_dir = os.path.join(os.getcwd(), "data", "scale_1", "512", "processed_mask")

In [12]:
import shutil

# Delete the folder if it already exists
if os.path.exists(new_mask_dir):
    shutil.rmtree(new_mask_dir)

os.makedirs(new_mask_dir)

### Converting masks

In [13]:
for filename in os.listdir(mask_dir):

    mask_path = os.path.join(mask_dir, filename)
    new_mask_path = os.path.join(new_mask_dir, filename)

    if filename.endswith(".png"):

        image = cv2.imread(mask_path)
            
        if filename.endswith("1.png"):

            # Dimensions of the white rectangle, having 1px black border
            rectangle_width, rectangle_height = image.shape[1] - 2, image.shape[0] - 2

            # Calculate the coordinates for the white rectangle
            x = (image.shape[1] - rectangle_width) // 2
            y = (image.shape[0] - rectangle_height) // 2

            cv2.rectangle(image, (x, y), (x+rectangle_width, y+rectangle_height), (255, 255, 255), -1)
            cv2.imwrite(new_mask_path, image)

        # Else if it ends with 0.png just save the same copy  
        original_image = image.copy()
        cv2.imwrite(new_mask_path, original_image)

### Feature extraction

In [14]:
# Initialize the feature extractor
extractor = featureextractor.RadiomicsFeatureExtractor()
extractor.enableAllFeatures()

print("Extraction parameters: ", extractor.enabledFeatures)

Extraction parameters:  {'firstorder': [], 'glcm': [], 'gldm': [], 'glrlm': [], 'glszm': [], 'ngtdm': [], 'shape': [], 'shape2D': []}


In [15]:
extractor.settings["force2D"] = True
print(extractor.settings)

{'minimumROIDimensions': 2, 'minimumROISize': None, 'normalize': False, 'normalizeScale': 1, 'removeOutliers': None, 'resampledPixelSpacing': None, 'interpolator': 'sitkBSpline', 'preCrop': False, 'padDistance': 5, 'distances': [1], 'force2D': True, 'force2Ddimension': 0, 'resegmentRange': None, 'label': 1, 'additionalInfo': True}


In [17]:
# Create data frame which will store feature values 
features_df = pd.DataFrame()

for filename in os.listdir(image_dir):

    image_path = os.path.join(image_dir, filename)
    mask_path = os.path.join(new_mask_dir, filename)

    features = extractor.execute(image_path, mask_path, label = 255)
    features_df = features_df.append(pd.Series(features), ignore_index = True)

Shape features are only available 3D input (for 2D input, use shape2D). Found 2D input
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
C:\Users\Korisnik\AppData\Local\Temp\ipykernel_12800\4041461289.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  features_df = features_df.append(pd.Series(features), ignore_index = True)
Shape features are only available 3D input (for 2D input, use shape2D). Found 2D input
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
C:\Users\Korisnik\AppData\Local\Temp\ipykernel_12800\4041461289.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  features_df = features_df.append(pd.Series(features), ignore_index = True)
Shape features are only available 3D input (for 2D input, use shape2D). Foun

In [ ]:
features_df.head(3)

,diagnostics_Versions_PyRadiomics,diagnostics_Versions_Numpy,diagnostics_Versions_SimpleITK,diagnostics_Versions_PyWavelet,diagnostics_Versions_Python,diagnostics_Configuration_Settings,diagnostics_Configuration_EnabledImageTypes,diagnostics_Image-original_Hash,diagnostics_Image-original_Dimensionality,diagnostics_Image-original_Spacing,...,original_glszm_SmallAreaHighGrayLevelEmphasis,original_glszm_SmallAreaLowGrayLevelEmphasis,original_glszm_ZoneEntropy,original_glszm_ZonePercentage,original_glszm_ZoneVariance,original_ngtdm_Busyness,original_ngtdm_Coarseness,original_ngtdm_Complexity,original_ngtdm_Contrast,original_ngtdm_Strength
0,v3.0.1,1.23.5,2.2.1,1.4.1,3.10.9,"{'minimumROIDimensions': 2, 'minimumROISize': ...",{'Original': {}},13f501ab5e4d2d2bee6d5303f0d25f463bef4733,2D,"(1.0, 1.0)",...,23.620827690568294,0.03847567467367387,4.638442559973614,0.5896260315729474,1.2198258855807722,58.90781930060185,0.00030747943438384225,27.441090616580958,0.03878475841848786,0.013273900668552878
1,v3.0.1,1.23.5,2.2.1,1.4.1,3.10.9,"{'minimumROIDimensions': 2, 'minimumROISize': ...",{'Original': {}},113ab59e2b6b19468a9ebeff444e5f207966df74,2D,"(1.0, 1.0)",...,6.496939633407332,0.12984349637178005,4.418469115363689,0.4766104602846956,3.198493437927262,707.1176368804727,3.897310837469989e-05,33.00420778631294,0.013954819475841223,0.004265080703148299
2,v3.0.1,1.23.5,2.2.1,1.4.1,3.10.9,"{'minimumROIDimensions': 2, 'minimumROISize': ...",{'Original': {}},b10f814af0cc2661a2f4609410eb3ae4edd35684,2D,"(1.0, 1.0)",...,6.642086491517708,0.10866769114223532,3.646060893729482,0.5400485436893204,2.047665698775407,27.05092346728035,0.0017974312438922239,8.431611791760808,0.023067855125242998,0.025505320798160164


In [ ]:
features_df.shape

(86, 115)